In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
import glob
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


log_file = 'SDTwittC_gender_1.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


files1 = glob.glob('benchmarks2/SDTwittC/Male/*')
files2 = glob.glob('benchmarks2/SDTwittC/Female/*')


# getting males comments
dfs = []
x = 0
for file in files1:    
    df = pd.read_csv(file, header=None, names=['text'], encoding='utf-8', sep='\t', quoting=3, engine='python')
    dfs.append(df)
    x += len(df)

dfm = pd.concat(dfs, ignore_index=True)
dfm.fillna('', inplace=True)
display(x)
display(len(dfm))
dfm = dfm[dfm['text'] != '']
display(len(dfm))
dfm['label'] = 0
display(dfm[:4])

# getting females comments
dfs = []
x = 0
for file in files2:    
    df = pd.read_csv(file, header=None, names=['text'], encoding='utf-8', sep='\t', quoting=3, engine='python')
    dfs.append(df)
    x += len(df)

dff = pd.concat(dfs, ignore_index=True)
dff.fillna('', inplace=True)
display(x)
display(len(dff))
dff = dff[dff['text'] != '']
display(len(dff))
dff['label'] = 1
display(dff[:4])


print()
df = pd.concat([dfm, dff], axis=0, ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
display(len(df))
display(df[:4])




classes_num = 2
display(classes_num)
display(len(df))
# display(len(df_test))


ds = Dataset.from_pandas(df)

ds = ds.train_test_split(test_size=0.2)
display(ds)

max_sequence_length = 128

models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib',
        'reemalyami/AraRoBERTa-SA',    
]
for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
        dataset_train = ds['train']
        dataset_validation = ds['test']                                                    
        
      

        def preprocess_function(examples):
            return tokenizer(examples['text'], truncation=True, padding="max_length",
                            max_length=max_sequence_length, add_special_tokens=True)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
       
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}


        
        
        epochs = 4
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 64
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 1000, #50_000
            evaluation_strategy = 'steps',
            # evaluate_during_training = True,
            eval_steps = 1000
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics = compute_metrics
        )
        
        
        # trainer.train(resume_from_checkpoint=True)
        trainer.train()
       

results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv('SDTwittC_gender_results_1.csv')
display(best_results)




2024-03-28 02:17:32.081036: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 02:17:32.106652: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 02:17:32.477940: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


233517

233517

233517

,text,label
0,حينما تخرج الحكمة من الحاكم الملك سلمان بن عبدالعزيز ❤,0
1,أعظم فيديو ترجمته في عام 2018 كان لهذا الرجل الي قال قصة حياته وتجربته في 8 دقائق فقط !! لكن كل دقيقة منها عظيمة و,0
2,اليوم في مباراة نهائي كأس خادم الحرمين الشريفين بين بيصير فيه طاقم كبير لتغطيته بأكثر من 580 كامير,0
3,نفس القوة هذا عرض خاص لمتابعين حسابي فقط,0


...

Map:   0%|          | 0/360664 [00:00<?, ? examples/s]

Map:   0%|          | 0/90166 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
1000,0.618900,0.599142,0.667502,0.666465
2000,0.589200,0.577177,0.684238,0.683698
3000,0.579300,0.560015,0.700818,0.697657
4000,0.569300,0.572939,0.702216,0.702072
5000,0.558300,0.570213,0.712508,0.711579
6000,0.533200,0.563324,0.714893,0.714299
7000,0.504300,0.553079,0.714904,0.713170
8000,0.499200,0.577935,0.717898,0.717807
9000,0.499200,0.566927,0.718708,0.718320
10000,0.496500,0.553262,0.721270,0.719000


Checkpoint destination directory bert/checkpoint-20000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.742819,0.742216
3,aubmindlab/bert-base-arabertv02-twitter,0.738782,0.738417
5,qarib/bert-base-qarib,0.733292,0.733278
8,reemalyami/AraRoBERTa-SA,0.725872,0.724648
